In [23]:
class Operad:
    def __init__(self):
        self.colors = {}
        self.operations = {}

    def add_color(self, name, prop={}):
        if name in self.colors.keys():
            t = self.colors[name]
            self.colors[name] = (t, prop)
        else:
            self.colors[name] = prop
        return name

    def add_color_for_trunk(self, name, prop={}):
        t = None
        if name in self.colors.keys():
            t = self.colors[name]
            self.colors[name] = (t, prop)
        else:
            self.colors[name] = prop
        return name, t

    def add_operation(self, name, prop={}):
        self.operations[name] = prop
        return name


class Tree:
    def __init__(self, operation, trunk, branches, operad):

        trunk, parent = operad.add_color_for_trunk(trunk, self)
        self.trunk = trunk
        if parent:
            parent.branches[trunk] = self
        self.branches = {
            operad.add_color(branch, self): uTree(branch) for branch in branches
        }
        self.node = operad.add_operation(operation, self)

    def print_edges(self, depth=0):
        return "".join(
            [f"{str(self.trunk)}\n"]
            + [
                "\t" * (depth + 1) + f"{branch.print_edges(depth + 1)}\n"
                for branch in self.branches.values()
            ]
        )

    def print_nodes(self, depth=0):
        return "".join(
            [f"{str(self.node)}\n"]
            + [
                "\t" * (depth + 1) + f"{branch.print_nodes(depth + 1)}\n"
                for branch in self.branches.values()
            ]
        )
    def __str__(self):
        return f"{self.node}({','.join(self.branches.keys())};{self.trunk})"


class uTree(Tree):
    def __init__(self, name):
        self.trunk = name
        self.branches = {}
        self.node = None

In [24]:
import re

operad = Operad()
operads = []

tree_str = "o0(c2,c1;c0)|o1(c3,c4;c1)|o2(c5,c6;c2)"
operations = tree_str.split("|")
regex = r"(o.*)\((.*)\)"
for operation in operations:
    match = re.match(regex, operation)
    if match:
        operation, parameters = match.groups()
        branches, trunk = parameters.split(";")
        operads.append(Tree(operation, trunk, branches.split(","), operad))

t = operads[0]
print(t.print_nodes())


o0
	o2
		None

		None


	o1
		None

		None





In [31]:
def string_to_tree_space(string):
    operad = Operad()
    operads = []
    operations = tree_str.split("|")
    regex = r"(.*)\((.*)\)"
    for operation in operations:
        match = re.match(regex, operation)
        if not match:
            raise RuntimeError(f"Operation not defined correctly {operation}")
        operation, parameters = match.groups()
        branches, trunk = parameters.split(";")
        operads.append(Tree(operation, trunk, branches.split(","), operad))
    return operads[0], operad


def tree_space_to_string(tree_space):
    _, operad = tree_space
    return "|".join(str(tree) for tree in operad.operations.values())


space = string_to_tree_space("o0(c2,c1;c0)|o1(c3,c4;c1)|o2(c5,c6;c2)")

print(tree_space_to_string(space))


o0(c2,c1;c0)|o1(c3,c4;c1)|o2(c5,c6;c2)
